<a href="https://colab.research.google.com/github/DyllanSchuster/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [29]:
# Import our dependencies
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
attrition_df.dtypes

,0
Age,int64
Attrition,object
BusinessTravel,object
Department,object
DistanceFromHome,int64
Education,int64
EducationField,object
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64


In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition','Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [14]:
#Preprocess y
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
dept_encoder = OneHotEncoder(sparse_output=False)
dept_encoded = dept_encoder.fit_transform(y_df[['Department']])
dept_columns = dept_encoder.get_feature_names_out(['Department'])
dept_encoded_df = pd.DataFrame(dept_encoded, columns=dept_columns)

attrition_encoder = LabelEncoder()
y_df['attrition_encoded'] = attrition_encoder.fit_transform(y_df['Attrition'])

# Concatenate the encoded columns to the original DataFrame
y_df_processed = pd.concat([y_df, dept_encoded_df ], axis=1)

# Drop the original "quality" and "color" columns
y_df_processed = y_df_processed.drop(['Attrition', 'Department'], axis=1)

y_df_processed.head()


<ipython-input-14-0fffb27c4a03>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_df['attrition_encoded'] = attrition_encoder.fit_transform(y_df['Attrition'])


,attrition_encoded,Department_Human Resources,Department_Research & Development,Department_Sales
0,1,0.0,0.0,1.0
1,0,0.0,1.0,0.0
2,1,0.0,1.0,0.0
3,0,0.0,1.0,0.0
4,0,0.0,1.0,0.0


In [16]:
#value counts of target variable department
y_df['Department'].value_counts() #Notice the imbalannce in the classes (departments)

,count
Department,
Research & Development,961
Sales,446
Human Resources,63


In [47]:
y_df["Attrition"].value_counts()

,count
Attrition,
No,1233
Yes,237


In [9]:
#Create y target variables for attrition and department

y_attrition = y_df_processed['attrition_encoded']
y_dept = y_df_processed[['Department_Human Resources',	'Department_Research & Development',	'Department_Sales']]

In [10]:
import random

In [18]:
# Create a list of at least 10 column names to use as X data
col_names = attrition_df.columns

# List of columns to exclude
exclude_columns = ['Attrition', 'Department']

# Filter out the columns to exclude
available_columns = [col for col in col_names if col not in exclude_columns]

# Randomly sample 10 column names from the remaining ones
random_columns = random.sample(available_columns, 10)

# Create a new dataframe with the randomly selected columns
random_df = attrition_df[random_columns]

# Show the data types for X_df
random_df.dtypes


,0
JobInvolvement,int64
HourlyRate,int64
BusinessTravel,object
OverTime,object
PerformanceRating,int64
YearsAtCompany,int64
JobSatisfaction,int64
StockOptionLevel,int64
TotalWorkingYears,int64
NumCompaniesWorked,int64


In [21]:
#Encode the object columns in the random dataframe to prepare for X (features) dataframe.

ohe = OneHotEncoder(sparse_output=False, drop='first')

# Identify object (categorical) columns
object_columns = random_df.select_dtypes(include=['object']).columns

# One-hot encode the categorical columns
if len(object_columns) > 0:
    encoded_data = ohe.fit_transform(random_df[object_columns])

    # Create a DataFrame for the encoded columns, using the one-hot encoder feature names
    encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(object_columns), index=random_df.index)

    # Drop the original categorical columns and concatenate the new encoded columns
    X = pd.concat([random_df.drop(columns=object_columns), encoded_df], axis=1)
else:
    # If no object columns, just set X_df as random_df
    X = random_df

# Display the resulting features dataframe
X.head()



,JobInvolvement,HourlyRate,PerformanceRating,YearsAtCompany,JobSatisfaction,StockOptionLevel,TotalWorkingYears,NumCompaniesWorked,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,OverTime_Yes
0,3,94,3,6,4,0,8,8,0.0,1.0,1.0
1,2,61,4,10,2,1,10,1,1.0,0.0,0.0
2,2,92,3,0,3,0,7,6,0.0,1.0,1.0
3,3,56,3,8,3,0,8,1,1.0,0.0,1.0
4,3,40,3,2,2,1,6,9,0.0,1.0,0.0


In [31]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_att_train, y_att_test, y_dept_train, y_dept_test = train_test_split(X,y_attrition,y_dept)

In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Did it above

In [34]:
# Create a StandardScaler
scaler = sklearn.preprocessing.StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create a OneHotEncoder for the Department column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data

#did it above


In [ ]:
# Create a OneHotEncoder for the Attrition column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data

#did it above

## Create, Compile, and Train the Model

In [41]:
# Find the number of columns in the X training data
input_features = len(X_train_scaled)

# Create the input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [42]:
# Create a branch for Department
# with a hidden layer and an output layer
dept_output = layers.Dense(3, activation='softmax', name='dept_output')(shared_layer2)
# Create the hidden layer


# Create the output layer



In [43]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(shared_layer2)

# Create the output layer



In [44]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'dept_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 11)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │            768 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 32)             │          2,080 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_output (Dense)       │ (None, 3)              │             99 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,980 (11.64 KB)

 Trainable params: 2,980 (11.64 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# Train the model
model.fit(
    X,
    {'dept_output': y_dept, 'attrition_output': y_attrition},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - attrition_output_accuracy: 0.6629 - dept_output_accuracy: 0.5159 - loss: 3.2089 - val_attrition_output_accuracy: 0.8333 - val_dept_output_accuracy: 0.5782 - val_loss: 1.3441
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8268 - dept_output_accuracy: 0.6424 - loss: 1.1904 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.5714 - val_loss: 1.2705
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8315 - dept_output_accuracy: 0.6460 - loss: 1.1834 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.5986 - val_loss: 1.2817
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8367 - dept_output_accuracy: 0.6700 - loss: 1.1814 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.5952 - val_loss: 1.2629
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8347 - de

In [53]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'dept_output': y_dept_test, 'attrition_output': y_att_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8348 - dept_output_accuracy: 0.6448 - loss: 1.1661


[1.136196494102478, 0.8505434989929199, 0.6630434989929199]

In [57]:
# Print the accuracy for both department and attrition
dept_output_accuracy = test_results[2]


attrition_output_accuracy = test_results[1]

print(f'Department predictions accuracy: {dept_output_accuracy:.2f}')
print(f'Attrition predictions accuracy: {attrition_output_accuracy:.2f}')

Department predictions accuracy: 0.66
Attrition predictions accuracy: 0.85


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?


2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?


YOUR ANSWERS HERE

1.  There is a class imbalance in the attrition set, specfically, more nos then yeses. Thus this neural network is not well trained to classify yeses as accurately as it can with nos. The same could be said for the what departmment the employee belongs to. Some departments are smaller than others, therefore the nn will not be good identifying these correctly. Because of the of equality between classes in the dataset accuracy isn't the best metric. However, because one of the outputs is a multiclass prediction, Cohen's Kappa score, is a good metric for evaluating multi-class classifiers on imbalanced datasets. It measures how close the predicted classes are to the actual classes compared to a random classification.
2. I used sigmoid for attrition output because it is a binary class between either yes, and no. Sigmoid is better for evaluating between two classes. For department classification softmax is more appropriate because it deals with multiclass classification. Since softmax divides the probability space equally for the different classes between 0 and 1, whereas, sigmoid divides only in to 2.
3. We could address the class imbalance of departments by including more employees in the less represented departments. If there are none maybe we include more employees from other departments, and aggregate similar departments to increase their presence in the dataset. We could create more fake employess that would be yes for attrition, to help the model identify them better.